In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from keras import Sequential

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.05G/1.06G [00:04<00:00, 263MB/s]
100% 1.06G/1.06G [00:04<00:00, 239MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
#generators
train_ds = keras.utils.image_dataset_from_directory(
            directory= "/content/train",
            labels='inferred',
            label_mode='int',
            batch_size=32,
            image_size=(256,256)
)
validation_ds =  keras.utils.image_dataset_from_directory(
            directory= "/content/test",
            labels='inferred',
            label_mode='int',
            batch_size=32,
            image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [8]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
#data augmentation layers
data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

# Apply data augmentation to the training dataset
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))


In [9]:
# Normalize
def process(image,label):
    image=tf.cast(image/255,tf.float32)
    return image,label
train_ds=train_ds.map(process)
validation_ds=validation_ds.map(process)

In [10]:
# create CNN model

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,847,297 (56.64 MB)

 Trainable params: 14,847,297 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(optimizer='SGD',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
history = model.fit(train_ds,validation_data=validation_ds,epochs=10)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 365s 570ms/step - accuracy: 0.5176 - loss: 0.6918 - val_accuracy: 0.5490 - val_loss: 0.6885
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 351s 561ms/step - accuracy: 0.5769 - loss: 0.6752 - val_accuracy: 0.5808 - val_loss: 0.6664
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 358s 572ms/step - accuracy: 0.6038 - loss: 0.6617 - val_accuracy: 0.6068 - val_loss: 0.6518
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 378s 567ms/step - accuracy: 0.6336 - loss: 0.6404 - val_accuracy: 0.6788 - val_loss: 0.6117
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 354s 566ms/step - accuracy: 0.6668 - loss: 0.6122 - val_accuracy: 0.6980 - val_loss: 0.5859
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 354s 566ms/step - accuracy: 0.6852 - loss: 0.5939 - val_accuracy: 0.7038 - val_loss: 0.5750
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 353s 564ms/step - accuracy: 0.6975 - loss: 0.5765 - val_accuracy: 0.6962 - val_loss: 0.5739
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 350s 560ms/step - accuracy: 0.7094 -

In [14]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(validation_ds)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


157/157 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - accuracy: 0.7344 - loss: 0.5372
Test Loss: 0.5398916006088257, Test Accuracy: 0.7328000068664551


In [15]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Function to predict new images
def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.  # Normalize the image
    prediction = model.predict(img_array)

    if prediction[0] > 0.5:
        print("It's a dog!")
    else:
        print("It's a cat!")

# Predict on a new image
predict_image(model, '/content/test/cats/cat.10.jpg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 854ms/step
It's a cat!


In [16]:

model.save('cats_vs_dogs_classifier.h5')



